In [ ]:
%matplotlib inline

import os
import nivapy3 as nivapy
import pandas as pd
import geopandas as gpd
import rasterio
import cmocean
import matplotlib.pyplot as plt
from rasterio.plot import show

plt.style.use('ggplot')

# DSToolkit PostGIS integration: accessing commonly used spatial datasets

This notebook provides a brief illustration of how to access spatial data via the DSToolkit. This functionality is very new/experimental - **feedback appreciated**! More detailed tutorials will be made available when I have time... :-)

**Note:** If you're looking for data so you can make maps using conventional desktop GIS (e.g. ArcGIS or QGIS), please see also the notebook `create_gis_package.ipynb`, which creates a data package (`.zip` archive) for your area of interest.

## 1. Connect to database

In [ ]:
eng = nivapy.da.connect_postgis()

## 2. Get fylke boundaries

List of layers is [here](https://github.com/NIVANorge/niva_jupyter_hub/blob/master/postgis_db/postgis_db_dataset_summary.md).

In [ ]:
# Read fylker
fyl_gdf = nivapy.da.read_postgis('cultural', 'norway_kartverket_fylker_2019_poly', eng)
fyl_gdf

In [ ]:
# Plot
fyl_gdf.plot(facecolor='white', 
              edgecolor='black', 
              figsize=(10,10),
             )

In [ ]:
# Focus on Aust-Agder
clip_gdf = fyl_gdf.query('fylkes_num == 9')

# Plot
clip_gdf.plot(facecolor='white', 
              edgecolor='black', 
              figsize=(10,10),
             )

## 3. Extract CORINE land cover (2018)

In [ ]:
# Read CORINE-2018 land cover, clipping to the specified fylke
gdf = nivapy.da.read_postgis('physical', 
                             'norway_nibio_corine_2018_poly', 
                             eng, 
                             clip=clip_gdf,
                            )

gdf.head()

In [ ]:
# Get top level class
gdf['main_class'] = gdf['klasse'].str[:3]

# Plot
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,10))

gdf.plot(column='main_class', 
         legend=True, 
         legend_kwds={'bbox_to_anchor':(1.2, 1)},
         ax=ax,
        )

clip_gdf.plot(edgecolor='k', 
              facecolor='none', 
              ax=ax,
              lw=2
             )

## 4. Extract 50 m DEM and hillshade

In [ ]:
# Clip raster to fylke
ras_list = [
            r'/home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_dem.tif',
            r'/home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_hillshade.tif',
           ]

for ras_path in ras_list:
    out_gtiff = os.path.split(ras_path)[1]
    out_gtiff = os.path.join('../examples/raster', out_gtiff)
    
    nivapy.spatial.clip_raster_to_gdf(ras_path,
                                      out_gtiff,
                                      clip_gdf,
                                      bounding_box=True,
                                     )

# Plot clipped version
dem = rasterio.open('../examples/raster/norway_kartverket_50m_dem.tif')
hil = rasterio.open('../examples/raster/norway_kartverket_50m_hillshade.tif')
    
fig, ax = plt.subplots(figsize=(10,10))
    
# Plot
cmap = cmocean.tools.crop_by_percent(cmocean.cm.topo, 50, which='min', N=None)
show(dem, ax=ax, cmap=cmap)
show(hil, ax=ax, alpha=0.6, cmap='gray')
    
# Plot mask
clip_gdf.plot(edgecolor='k', 
              facecolor='none', 
              ax=ax,
              lw=2
             )

dem.close()
hil.close()